# import packages

In [21]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os.path
import numpy as np
import datetime
import os
import geopy
import folium
from scipy import stats

# read the data

## paths and headers

In [22]:
root_path = os.getcwd()
paths = []
users = []
users_list = ['User1','User2']
for user in users_list:
    for root, dirs, files in os.walk(root_path + '/data/' + user):
        for directory in dirs: 
            paths.append(root + '/' + directory +'/')
            users.append(user)

#########################################################################
location_headers = ['time', 'ignore1', 'ignore2', 'accuracy', 'lat', 'long', 'HAGL'] 
# HAGL stand for height above ground level
motion_headers = ['time','accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ', 
                 'magX', 'magY', 'magZ', 'oriW', 'oriX', 'oriY', 'oriZ', 
                 'gravX', 'gravY', 'gravZ', 'LaccX', 'LaccY', 'LaccZ', 
                 'pressure', 'HAGL_P', 'temp']
labels_track_main_headers = ['start','end','label']

# to group mode types
#modes = ['still','walk','run','bike','car','bus','train','subway','unlabelled'] # [0,...,8]
#match = [[0,1,2,3],[4,5],[6],[7],[8,9],[10,11,12,13],[14,15],[16,17]]
#mode_dict = {0:0, 1:0, 2:0, 3:0, 4:1, 5:1, 6:2, 7:3, 8:4, 9:4, 10:5, 11:5, 12:5, 13:5, 14:6, 15:6, 16:7, 17:7}
mode_dict={'Still':[0,1,2,3],'Walking':[4,5],'Run':[6],'Bike':[7],
           'Car':[8,9],'Bus':[10,11,12,13],'Train':[14,15],'Subway':[16,17]}

# to gather data
all_data = pd.DataFrame()
all_user_info = pd.DataFrame()


In [3]:
#for path in paths[:len(paths)]:
#    print('\nPath:',path)
#    ################################read
#    #user_info = { line.split(': ')[0] : line.split(': ')[1][:-1] for line in open(path+'00inf.txt') }
#    location = pd.read_csv(path + 'Torso_Location.txt', delimiter = " ",names = location_headers, index_col=0)
#    #motion = pd.read_csv(path + 'Torso_Motion.txt', delimiter = " ", names = motion_headers, index_col=0)
#    #labels_track_main = pd.read_csv(path + 'labels_track_main.txt', delimiter = " ", names = labels_track_main_headers)
#    print(location)
#    #################################drop

## input the data

In [78]:
all_data = pd.DataFrame()
all_user_info = pd.DataFrame()

for path in paths[:len(paths)]:
    print('\nPath:',path)
    ################################read
    user_info = { line.split(': ')[0] : line.split(': ')[1][:-1] for line in open(path+'00inf.txt') }
    location = pd.read_csv(path + 'Torso_Location.txt', delimiter = " ",names = location_headers, index_col=0)
    motion = pd.read_csv(path + 'Torso_Motion.txt', delimiter = " ", names = motion_headers, index_col=0)
    labels_track_main = pd.read_csv(path + 'labels_track_main.txt', delimiter = " ", names = labels_track_main_headers)
    #################################drop
    location = location.drop(['ignore1','ignore2'],axis=1)
    motion['gyr'] = np.linalg.norm(motion[['gyrX','gyrY','gyrZ']],axis=1)
    motion['mag'] = np.linalg.norm(motion[['magX','magY','magZ']],axis=1)
    motion['grav'] = np.linalg.norm(motion[['gravX','gravY','gravZ']],axis=1)
    to_drop = motion.columns[[0,1,2,3,4,5,6,7,8,13,14,15,20]] # linacc = 16,17,18,
    motion = motion.drop(to_drop,axis=1)
    #############################merge motion and location
    motion.set_index(motion.index.values.astype(int))
    #location.set_index(location.index.values.astype(int))
    data = pd.merge(motion, location, left_index=True, right_index=True, how="left")
    data = data.dropna(how='any') # 'all'
    for index,row in labels_track_main.iterrows():
        mode_number=row[2]
        for keys in mode_dict:
            if mode_number in mode_dict[keys]:
                labels_track_main.loc[index,'label']=keys

    #labels_track_main.label = labels_track_main['label'].map(mode_dict) 

    
    ##############################labels_track_main and motion
    data['mode'] = 'unlabelled'
    data['tripNr'] = float(user_info['recid'])+0.001*(len(labels_track_main))
    data['User'] = None
    data['recid'] = None
    for index,row in labels_track_main.iterrows():
        start = row[0]
        end = row[1]
        label = row[2]
        trip_time_point = (data.index > start) & (data.index <= end)
        data.loc[trip_time_point,'mode'] = label
        data.loc[trip_time_point,'tripNr'] = float(user_info['recid'])+0.001*(index)
    data['User'] = user_info['sessionid']
    data['recid'] = user_info['recid']
    all_data = pd.concat([all_data,data])
    user_info = pd.DataFrame([user_info])
    all_user_info = pd.concat([all_user_info,user_info])
    
    #print('Length before (%s) vs after merging & droping nan (%s)'%(length_init,len(all_data)))
#all_user_info = pd.DataFrame(all_user_info)
#all_data = pd.DataFrame(all_data)
#all_data = pd.concat([all_data])  


Path: /Users/lepeng/Repositories/MDA/MDA0208/data/User1/220617/

Path: /Users/lepeng/Repositories/MDA/MDA0208/data/User1/260617/

Path: /Users/lepeng/Repositories/MDA/MDA0208/data/User1/270617/

Path: /Users/lepeng/Repositories/MDA/MDA0208/data/User2/140717/

Path: /Users/lepeng/Repositories/MDA/MDA0208/data/User2/140617/

Path: /Users/lepeng/Repositories/MDA/MDA0208/data/User2/180717/


In [79]:
#all_data['datetime'] = all_data.index.to_series().apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0))
#display(all_data['datetime'])
#all_data['datetime'] = pd.to_datetime(all_data.index.to_series(),unit='ms')
all_data['datetime'] = all_data.index.to_series()
all_data['datetime'] = pd.to_datetime(all_data['datetime'],unit='ms')
#all_data['datetime']
#all_data = all_data.drop_duplicates(subset=['datetime'])
#all_data = all_data.reset_index(all_data.datetime)

all_data = all_data.reset_index()


#all_data = all_data.set_index(all_data.datetime)
#all_data

In [80]:
all_data

,time,oriW,oriX,oriY,oriZ,LaccX,LaccY,LaccZ,pressure,temp,...,grav,accuracy,lat,long,HAGL,mode,tripNr,User,recid,datetime
0,1.498121e+12,0.550023,0.449164,-0.564783,-0.420408,-0.595957,-1.372849,0.173059,1005.107631,0.00000,...,9.806549,48.0,50.846986,-0.133405,116.693999,unlabelled,220617.02,User1,220617,2017-06-22 08:46:43.390000128
1,1.498121e+12,0.711295,0.667593,-0.021792,0.218854,0.164321,6.407377,-1.372446,1005.113749,0.00000,...,9.806535,48.0,50.846965,-0.133387,118.290886,unlabelled,220617.02,User1,220617,2017-06-22 08:46:46.620000000
2,1.498121e+12,0.704656,0.686372,-0.113010,0.139928,-0.948191,-4.295378,-0.132671,1005.145165,0.00000,...,9.806603,48.0,50.846894,-0.133366,118.368997,Walking,220617.00,User1,220617,2017-06-22 08:46:52.430000128
3,1.498121e+12,0.717173,0.672209,-0.103821,0.151705,-0.670852,-4.799349,-0.619527,1005.110957,0.00000,...,9.806549,48.0,50.846889,-0.133378,118.368997,Walking,220617.00,User1,220617,2017-06-22 08:46:53.430000128
4,1.498121e+12,0.729197,0.662254,-0.089967,0.146949,-1.006420,-2.853471,0.186316,1005.147842,0.00000,...,9.806537,48.0,50.846780,-0.133376,118.455036,Walking,220617.00,User1,220617,2017-06-22 08:46:57.440000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16037,1.500391e+12,0.398715,-0.005248,0.010691,-0.916997,0.127777,-0.010927,-0.020098,1011.351652,33.46875,...,9.806584,24.0,51.083063,0.756653,88.297068,unlabelled,180717.02,User2,180717,2017-07-18 15:08:24.720000000
16038,1.500391e+12,0.395846,-0.011089,0.008797,-0.918207,-0.003984,0.000434,-0.010000,1011.356450,33.40625,...,9.806629,24.0,51.083061,0.756651,88.297068,unlabelled,180717.02,User2,180717,2017-07-18 15:08:27.720000000
16039,1.500391e+12,0.395932,-0.011232,0.008839,-0.918168,-0.006400,0.011500,0.009258,1011.333291,33.15625,...,9.806604,48.0,51.083050,0.756637,88.314025,unlabelled,180717.02,User2,180717,2017-07-18 15:08:43.760000000
16040,1.500391e+12,0.395975,-0.011271,0.008755,-0.918149,0.011910,0.009915,-0.009600,1011.347102,33.09375,...,9.806592,48.0,51.083049,0.756638,88.314025,unlabelled,180717.02,User2,180717,2017-07-18 15:08:50.809999872


In [81]:
all_data = all_data.set_index(all_data.datetime)
all_data

,time,oriW,oriX,oriY,oriZ,LaccX,LaccY,LaccZ,pressure,temp,...,grav,accuracy,lat,long,HAGL,mode,tripNr,User,recid,datetime
datetime,,,,,,,,,,,,,,,,,,,,,
2017-06-22 08:46:43.390000128,1.498121e+12,0.550023,0.449164,-0.564783,-0.420408,-0.595957,-1.372849,0.173059,1005.107631,0.00000,...,9.806549,48.0,50.846986,-0.133405,116.693999,unlabelled,220617.02,User1,220617,2017-06-22 08:46:43.390000128
2017-06-22 08:46:46.620000000,1.498121e+12,0.711295,0.667593,-0.021792,0.218854,0.164321,6.407377,-1.372446,1005.113749,0.00000,...,9.806535,48.0,50.846965,-0.133387,118.290886,unlabelled,220617.02,User1,220617,2017-06-22 08:46:46.620000000
2017-06-22 08:46:52.430000128,1.498121e+12,0.704656,0.686372,-0.113010,0.139928,-0.948191,-4.295378,-0.132671,1005.145165,0.00000,...,9.806603,48.0,50.846894,-0.133366,118.368997,Walking,220617.00,User1,220617,2017-06-22 08:46:52.430000128
2017-06-22 08:46:53.430000128,1.498121e+12,0.717173,0.672209,-0.103821,0.151705,-0.670852,-4.799349,-0.619527,1005.110957,0.00000,...,9.806549,48.0,50.846889,-0.133378,118.368997,Walking,220617.00,User1,220617,2017-06-22 08:46:53.430000128
2017-06-22 08:46:57.440000000,1.498121e+12,0.729197,0.662254,-0.089967,0.146949,-1.006420,-2.853471,0.186316,1005.147842,0.00000,...,9.806537,48.0,50.846780,-0.133376,118.455036,Walking,220617.00,User1,220617,2017-06-22 08:46:57.440000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-07-18 15:08:24.720000000,1.500391e+12,0.398715,-0.005248,0.010691,-0.916997,0.127777,-0.010927,-0.020098,1011.351652,33.46875,...,9.806584,24.0,51.083063,0.756653,88.297068,unlabelled,180717.02,User2,180717,2017-07-18 15:08:24.720000000
2017-07-18 15:08:27.720000000,1.500391e+12,0.395846,-0.011089,0.008797,-0.918207,-0.003984,0.000434,-0.010000,1011.356450,33.40625,...,9.806629,24.0,51.083061,0.756651,88.297068,unlabelled,180717.02,User2,180717,2017-07-18 15:08:27.720000000
2017-07-18 15:08:43.760000000,1.500391e+12,0.395932,-0.011232,0.008839,-0.918168,-0.006400,0.011500,0.009258,1011.333291,33.15625,...,9.806604,48.0,51.083050,0.756637,88.314025,unlabelled,180717.02,User2,180717,2017-07-18 15:08:43.760000000


In [82]:
gdf_all_data = gpd.GeoDataFrame(data=all_data, geometry=gpd.points_from_xy(all_data.lat, all_data.long))

#gdf_all_data = gdf_all_data.set_index(gdf_all_data.datetime)
#gdf_all = gdf_all.set_index(gdf_all.time_dt)

/Users/lepeng/opt/anaconda3/lib/python3.9/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


# data processing

## functions

In [83]:
def lat_lon_2_m(latitude_1, longitude_1, latitude_2, longitude_2):
    
    # Radius of the earth in m
    radius_earth = 6371009

    d_latitude = np.deg2rad(latitude_2 - latitude_1)
    d_longitude = np.deg2rad(longitude_2 - longitude_1)
    latitude_1 = np.deg2rad(latitude_1)
    latitude_2 = np.deg2rad(latitude_2)

    a = (np.sin(d_latitude / 2)) ** 2 + np.cos(latitude_1) * np.cos(latitude_2) * (np.sin(d_longitude / 2)) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = radius_earth * c

    return distance

def calculate_distance(latitude, longitude):
    if len(latitude) != len(longitude):
        return np.nan
    else:
        # initialize distance array
        distance = np.zeros(len(latitude))
        # loop latitude, longitude
        for i in range(0, len(latitude) - 1):
            # calculate distance between two consecutive points using lat_lon_2_km function
            d = lat_lon_2_m(
                latitude[i],
                longitude[i],
                latitude[i + 1],
                longitude[i + 1])
            # append distance to array
            distance[i + 1] = d
        return distance
    
def calculate_speed(array_distances, array_deltatime):
    if len(array_distances) != len(array_deltatime):
        print("input vector length does not match!")
        return np.nan
    else:
        # preallocate the return variable
        speed = [0] * (len(array_distances)-1)
        for i in range(0, len(array_distances)-1):
            if array_deltatime[i] != 0:
                speed[i] = array_distances[i] / array_deltatime[i] * 3.6
            elif array_distances[i] == 0:
                speed[i] = 0
            else: 
                speed[i] = speed[i-1]
        # Add entry to acceleration, to have same length as speed
        speed = np.append(speed, [0])
        return speed
    
def calculate_acceleration(speed, array_deltatime):
    acceleration = np.diff(speed)
    for i in range(0, len(array_deltatime)-1):
        if array_deltatime[i] != 0:
            acceleration[i] /= array_deltatime[i]
        else: 
            acceleration[i] = acceleration[i-1]
    acceleration = np.append(acceleration, [0])
    return acceleration

def correct_speed(speed, acceleration, threshold):
    if len(speed) != len(acceleration):
        print("input vector length does not match!")
        return np.nan
    else:
        # Initiate the corrected speed vector with the original
        speed_corr = np.zeros((len(speed), 1))
        # Loop over and replace speeds, if acceleration > threshold
        speed_corr = speed
        for i in range(0, len(speed)-3):
            #speed_corr[i] = speed[i]
            if abs(acceleration[i]) > threshold:
                #speed_corr[i:i+3] = np.NaN
                speed_corr[i-3:i+3] = np.NaN  
        return speed_corr
    
def movingaverage (values, window):
    weights = np.repeat(1.0, window)/window
    sma = np.convolve(values, weights, 'same')
    return sma

def calculate_rpa(distances,delta_times,speed,acc):
    total_distance = distances.sum()
    flag = 0
    for i in range(len(delta_times)):
        if not np.isnan(acc[i]) and not np.isnan(speed[i]) and acc[i]>0:
            flag += speed[i] * acc[i] * delta_times[i]
    rpa = flag/total_distance
    return rpa

## new colmuns

In [84]:
gdf = gdf_all_data
gdf['d_dist'] = None
gdf['d_time'] = None
gdf['vel'] = None
gdf['acc'] = None
gdf['vel_corr'] = None
#gdf_all['vel_ma'] = None
for tripNr in all_data.tripNr.unique():
    
    # gather & prep
    tripdata = gdf_all_data[gdf_all_data.tripNr == tripNr]

    THRESHOLD_ACC = 5
    
    # distance & time
    distances = calculate_distance(tripdata.geometry.x, tripdata.geometry.y)
    delta_times = tripdata.index.to_series().diff().apply(lambda x: x.total_seconds())
    
    # velocity & acc
    speed = calculate_speed(distances, delta_times)
    acceleration = calculate_acceleration(speed, delta_times)
    
    # vel_corr & vel_ma
    speed_ma = movingaverage(speed, 2)
    speed_corr = correct_speed(speed, acceleration, THRESHOLD_ACC)
    
    # assign
    timepoint = (gdf_all_data.tripNr==tripNr)
    gdf.loc[timepoint,'d_dist'] = distances
    gdf.loc[timepoint,'vel'] = speed
    gdf.loc[timepoint,'acc'] = acceleration
    gdf.loc[timepoint,'vel_corr'] = speed_corr
    gdf.loc[timepoint,'d_time'] = delta_times
    #gdf_all.loc[mask,'vel_ma'] = speed_ma

gdf['d_dist'] = pd.to_numeric(gdf['d_dist'])
gdf['d_time'] = pd.to_numeric(gdf['d_time'])
gdf['vel'] = pd.to_numeric(gdf['vel'])
gdf['acc'] = pd.to_numeric(gdf['acc'])
gdf['vel_corr'] = pd.to_numeric(gdf['vel_corr'])
#gdf['vel_ma'] = pd.to_numeric(gdf['vel_ma'])
#
print('New columns:')
display(gdf.iloc[:10,-5:])
display(gdf.iloc[:10,:][['mode','long','lat','d_dist','d_time']])

New columns:


,d_dist,d_time,vel,acc,vel_corr
datetime,,,,,
2017-06-22 08:46:43.390000128,0.000000,NaN,NaN,NaN,NaN
2017-06-22 08:46:46.620000000,2.664046,3.229999,2.969217,0.285415,2.969217
2017-06-22 08:46:52.430000128,0.000000,NaN,NaN,NaN,NaN
2017-06-22 08:46:53.430000128,1.026499,1.000000,3.695396,7.181507,3.695396
2017-06-22 08:46:57.440000000,12.115659,4.009999,10.876903,-1.129117,10.876903
2017-06-22 08:47:03.449999872,10.599545,6.009999,6.349146,0.012140,6.349146
2017-06-22 08:47:21.480000000,32.164061,18.030000,6.422109,-0.033330,6.422109
2017-06-22 08:47:28.489999872,11.335122,7.009999,5.821176,0.207548,5.821176
2017-06-22 08:47:46.520000000,36.441086,18.030000,7.276090,-0.177650,7.276090


,mode,long,lat,d_dist,d_time
datetime,,,,,
2017-06-22 08:46:43.390000128,unlabelled,-0.133405,50.846986,0.000000,NaN
2017-06-22 08:46:46.620000000,unlabelled,-0.133387,50.846965,2.664046,3.229999
2017-06-22 08:46:52.430000128,Walking,-0.133366,50.846894,0.000000,NaN
2017-06-22 08:46:53.430000128,Walking,-0.133378,50.846889,1.026499,1.000000
2017-06-22 08:46:57.440000000,Walking,-0.133376,50.846780,12.115659,4.009999
2017-06-22 08:47:03.449999872,Walking,-0.133410,50.846687,10.599545,6.009999
2017-06-22 08:47:21.480000000,Walking,-0.133393,50.846398,32.164061,18.030000
2017-06-22 08:47:28.489999872,Walking,-0.133326,50.846305,11.335122,7.009999
2017-06-22 08:47:46.520000000,Walking,-0.132889,50.846482,36.441086,18.030000


In [89]:
gdf = gdf.drop(['datetime'], axis=1)
gdf

,time,oriW,oriX,oriY,oriZ,LaccX,LaccY,LaccZ,pressure,temp,...,mode,tripNr,User,recid,geometry,d_dist,d_time,vel,acc,vel_corr
datetime,,,,,,,,,,,,,,,,,,,,,
2017-06-22 08:46:43.390000128,1.498121e+12,0.550023,0.449164,-0.564783,-0.420408,-0.595957,-1.372849,0.173059,1005.107631,0.00000,...,unlabelled,220617.02,User1,220617,POINT (50.84699 -0.13341),0.000000,NaN,NaN,NaN,NaN
2017-06-22 08:46:46.620000000,1.498121e+12,0.711295,0.667593,-0.021792,0.218854,0.164321,6.407377,-1.372446,1005.113749,0.00000,...,unlabelled,220617.02,User1,220617,POINT (50.84696 -0.13339),2.664046,3.229999,2.969217,0.285415,2.969217
2017-06-22 08:46:52.430000128,1.498121e+12,0.704656,0.686372,-0.113010,0.139928,-0.948191,-4.295378,-0.132671,1005.145165,0.00000,...,Walking,220617.00,User1,220617,POINT (50.84689 -0.13337),0.000000,NaN,NaN,NaN,NaN
2017-06-22 08:46:53.430000128,1.498121e+12,0.717173,0.672209,-0.103821,0.151705,-0.670852,-4.799349,-0.619527,1005.110957,0.00000,...,Walking,220617.00,User1,220617,POINT (50.84689 -0.13338),1.026499,1.000000,3.695396,7.181507,3.695396
2017-06-22 08:46:57.440000000,1.498121e+12,0.729197,0.662254,-0.089967,0.146949,-1.006420,-2.853471,0.186316,1005.147842,0.00000,...,Walking,220617.00,User1,220617,POINT (50.84678 -0.13338),12.115659,4.009999,10.876903,-1.129117,10.876903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-07-18 15:08:24.720000000,1.500391e+12,0.398715,-0.005248,0.010691,-0.916997,0.127777,-0.010927,-0.020098,1011.351652,33.46875,...,unlabelled,180717.02,User2,180717,POINT (51.08306 0.75665),3.565671,12.060000,NaN,-0.067721,NaN
2017-07-18 15:08:27.720000000,1.500391e+12,0.395846,-0.011089,0.008797,-0.918207,-0.003984,0.000434,-0.010000,1011.356450,33.40625,...,unlabelled,180717.02,User2,180717,POINT (51.08306 0.75665),0.206385,3.000000,NaN,0.034361,NaN
2017-07-18 15:08:43.760000000,1.500391e+12,0.395932,-0.011232,0.008839,-0.918168,-0.006400,0.011500,0.009258,1011.333291,33.15625,...,unlabelled,180717.02,User2,180717,POINT (51.08305 0.75664),1.562757,16.040000,0.350743,-0.017146,0.350743


# save to the files

In [90]:
gdf.to_file("gdf_all_data.geojson", driver="GeoJSON")  
all_data.to_csv("all_data.csv")  